# LangChain을 활용한 RAG (Retrieval-Augmented Generation) 시스템

이 노트북은 PDF 문서를 로드하고, 검색 기반 생성(RAG)을 수행하는 전체 파이프라인을 구현합니다.

## RAG의 주요 단계:
1. **데이터 로드**: PDF 파일을 로드하고 텍스트 추출
2. **전처리**: 텍스트를 청크로 분할하고 정제
3. **임베딩**: 텍스트를 벡터로 변환하여 벡터 DB에 저장
4. **LLM 연결**: GPT 모델과 연결
5. **프롬프트 엔지니어링**: 검색된 문서와 질문을 결합한 프롬프트 작성
6. **실행**: 질문에 대한 답변 생성


In [ ]:
pip install langchain langchain-openai langchain-community chromadb pypdf

In [ ]:
# 필요한 라이브러리 임포트
# 먼저 필요한 패키지가 설치되어 있는지 확인합니다

from langchain_community.document_loaders import PyPDFLoader  # PDF 파일 로드용
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 텍스트 분할용
from langchain_openai import OpenAIEmbeddings, ChatOpenAI  # OpenAI 임베딩 및 LLM
from langchain_community.vectorstores import Chroma  # 벡터 데이터베이스
from langchain.chains import RetrievalQA  # RAG 체인
from langchain.prompts import PromptTemplate  # 프롬프트 템플릿
import chromadb  # chromadb 직접 import 테스트
import os


In [ ]:
# ============================================
# 1단계: 데이터 로드 (PDF)
# ============================================

# OpenAI API 키 설정 (환경 변수에서 가져오거나 직접 설정)
# 방법 1: 환경 변수에서 가져오기 (권장)
os.environ["OPENAI_API_KEY"] = "sk-"

# PDF 파일 경로 설정
pdf_path = "SPRi AI Brief_10월호_산업동향_1002_F.pdf"

# PyPDFLoader를 사용하여 PDF 파일 로드
# PyPDFLoader는 PDF의 각 페이지를 Document 객체로 변환합니다
loader = PyPDFLoader(pdf_path)

# PDF에서 모든 페이지의 텍스트를 추출하여 Document 리스트로 반환
# 각 Document는 page_content(텍스트 내용)와 metadata(페이지 번호 등)를 포함합니다
documents = loader.load()

# 로드된 문서 정보 출력
print(f"총 {len(documents)}개의 페이지가 로드되었습니다.")
print(f"첫 번째 페이지의 내용 길이: {len(documents[0].page_content)} 문자")
print(f"첫 번째 페이지 미리보기:\n{documents[0].page_content[:200]}...")


총 29개의 페이지가 로드되었습니다.
첫 번째 페이지의 내용 길이: 23 문자
첫 번째 페이지 미리보기:
2025년10월호인공지능 산업의 최신 동향...


In [ ]:
# ============================================
# 2단계: 전처리 (텍스트 분할 및 정제)
# ============================================

# RecursiveCharacterTextSplitter 초기화
# 이 분할기는 텍스트를 의미 있는 청크로 나누는 역할을 합니다
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 각 청크의 최대 문자 수
    # 청크가 1000자를 넘으면 자동으로 분할합니다
    chunk_overlap=200,  # 청크 간 겹치는 문자 수
    # 이전 청크의 마지막 200자와 다음 청크의 처음 200자가 겹칩니다
    # 이렇게 하면 문맥이 끊기지 않고 연결됩니다
    length_function=len,  # 길이를 측정하는 함수 (문자 수 기준)
    separators=["\n\n", "\n", " ", ""]  # 분할 우선순위
    # 먼저 두 줄 바꿈으로 분할하고, 안되면 한 줄 바꿈, 공백, 글자 단위로 분할합니다
)

# 문서를 청크로 분할
# 각 Document 객체가 여러 개의 작은 Document로 분할됩니다
texts = text_splitter.split_documents(documents)

# 분할 결과 확인
print(f"원본 문서 수: {len(documents)}")
print(f"분할된 청크 수: {len(texts)}")
print(f"\n첫 번째 청크의 길이: {len(texts[0].page_content)} 문자")
print(f"첫 번째 청크 미리보기:\n{texts[0].page_content[:300]}...")
print(f"\n첫 번째 청크의 메타데이터: {texts[0].metadata}")


원본 문서 수: 29
분할된 청크 수: 70

첫 번째 청크의 길이: 23 문자
첫 번째 청크 미리보기:
2025년10월호인공지능 산업의 최신 동향...

첫 번째 청크의 메타데이터: {'producer': 'Hancom PDF 1.3.0.505', 'creator': 'Hancom PDF 1.3.0.505', 'creationdate': '2025-10-02T12:57:42+09:00', 'author': 'dj', 'moddate': '2025-10-02T12:57:42+09:00', 'pdfversion': '1.4', 'source': 'SPRi AI Brief_10월호_산업동향_1002_F.pdf', 'total_pages': 29, 'page': 0, 'page_label': '1'}


In [ ]:
# ============================================
# 3단계: 임베딩 (텍스트를 벡터로 변환)
# ============================================

# 1. OpenAI 임베딩 모델 초기화
# (참고: API 키는 os.getenv("OPENAI_API_KEY")를 통해 환경 변수에서 자동으로 읽어옵니다)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
print("✓ OpenAI 임베딩 모델 초기화 완료")


# 2. Chroma 벡터 데이터베이스 생성
# 'texts' 변수는 이전 단계에서 분할된 문서 리스트라고 가정합니다.
vectorstore = Chroma.from_documents(
    documents=texts,                  # 분할된 문서들
    embedding=embeddings,             # 사용할 임베딩 모델
    persist_directory="./chroma_db"   # 벡터 DB 저장 경로
)

# 벡터 DB가 성공적으로 생성되었는지 확인
# len(texts)를 확인하기 위해 'texts' 변수가 필요합니다.
# 만약 'texts' 변수가 없다면 이 줄은 주석 처리하거나 삭제해야 합니다.
print(f"✓ 벡터 데이터베이스에 {len(texts)}개의 문서가 저장되었습니다.")
print("✓ 벡터 DB 경로: ./chroma_db")


# 3. 검색 테스트 (선택사항)
query = "주요 내용은 무엇인가요?"
results = vectorstore.similarity_search(query, k=2) # 상위 2개 유사 문서 검색

print(f"\n✓ 검색 테스트 성공 - 질문: '{query}'")
print(f"✓ 검색된 문서 수: {len(results)}")

if len(results) > 0:
    print(f"✓ 첫 번째 검색 결과 미리보기:\n{results[0].page_content[:200]}...")
else:
    print("⚠ 검색 결과가 없습니다.")

✓ OpenAI 임베딩 모델 초기화 완료
✓ 벡터 데이터베이스에 70개의 문서가 저장되었습니다.
✓ 벡터 DB 경로: ./chroma_db

✓ 검색 테스트 성공 - 질문: '주요 내용은 무엇인가요?'
✓ 검색된 문서 수: 2
✓ 첫 번째 검색 결과 미리보기:
정책･법제기업･산업기술･연구인력･교육
3
대만 행정원, AI 기본법 통과 후 입법원에 제출n대만 행정원이 AI 개발과 응용에 우호적 환경을 조성해 대만을 글로벌 AI 개발의 핵심 거점으로 만들기 위한 「AI 기본법」을 가결하고 입법원에 제출n법안은 AI 개발과 응용의 기본 원칙과 정부의 역할과 의무, AI 위험관리 방안을 포괄하며, 기술 혁신과 국제협력에 ...


In [ ]:
# ============================================
# 4단계: LLM 연결 (GPT 모델 연결)
# ============================================

# 1. ChatOpenAI를 사용하여 GPT 모델 초기화
# (API 키는 환경 변수 OPENAI_API_KEY에서 자동으로 읽어옵니다)
llm = ChatOpenAI(
    model_name="gpt-5",  # 사용할 GPT 모델
    temperature=0.1               # 창의성 정도 (0.0 ~ 1.0)
)
print("✓ ChatOpenAI 모델 초기화 완료 (gpt-5)")

# 2. LLM 연결 테스트
# .invoke()를 사용하여 모델에 직접 질문(프롬프트)을 전달합니다.
test_response = llm.invoke("안녕하세요! 간단히 자기소개 해주세요.")

print("✓ LLM 연결 테스트 성공!")
# .content 속성에 모델의 실제 응답 내용이 들어있습니다.
print(f"응답: {test_response.content}")

✓ ChatOpenAI 모델 초기화 완료 (gpt-5)
✓ LLM 연결 테스트 성공!
응답: 안녕하세요! 저는 ChatGPT, OpenAI가 만든 대화형 인공지능입니다. 질문에 답변하고, 글쓰기·요약·번역, 아이디어 브레인스토밍, 학습 보조, 간단한 코딩과 데이터 정리까지 도와드릴 수 있어요. 한국어로 편하게 말씀해 주세요. 지금 어떤 도움을 드릴까요?


In [ ]:
# ============================================
# 5단계: 프롬프트 엔지니어링
# ============================================

# 커스텀 프롬프트 템플릿 생성
custom_prompt_template = """다음 컨텍스트를 사용하여 질문에 답변해주세요.
만약 답을 모르겠다면 모른다고 말하고, 답을 만들어내려고 하지 마세요.
답변은 한국어로 작성해주세요.

컨텍스트: {context}

질문: {question}

답변:"""

# PromptTemplate 객체 생성
# {context}와 {question}은 나중에 실제 값으로 대체됩니다
PROMPT = PromptTemplate(
    template=custom_prompt_template,
    input_variables=["context", "question"]  # 프롬프트에 사용될 변수들
)

# 프롬프트 미리보기
print("프롬프트 템플릿:")
print(custom_prompt_template)
print("\n프롬프트 엔지니어링 완료!")

프롬프트 템플릿:
다음 컨텍스트를 사용하여 질문에 답변해주세요.
만약 답을 모르겠다면 모른다고 말하고, 답을 만들어내려고 하지 마세요.
답변은 한국어로 작성해주세요.

컨텍스트: {context}

질문: {question}

답변:

프롬프트 엔지니어링 완료!


In [ ]:
# ============================================
# 6단계: RAG 체인 구성
# ============================================

# RetrievalQA 체인 생성
# 이 체인은 다음 작업을 순차적으로 수행합니다:
# 1. 사용자 질문을 받음
# 2. 벡터 DB에서 관련 문서 검색 (Retrieval)
# 3. 검색된 문서를 컨텍스트로 사용하여 프롬프트 작성 ('PROMPT' 템플릿 사용)
# 4. LLM에 프롬프트를 전달하여 답변 생성 (Generation)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,                          # 4단계에서 정의한 LLM
    chain_type="stuff",               # "stuff": 검색된 문서를 모두 프롬프트에 넣음
    retriever=vectorstore.as_retriever(
        search_kwargs={"k": 3}        # 상위 3개의 관련 문서를 검색
    ),
    return_source_documents=True,     # 답변의 근거가 된 원본 문서 반환
    chain_type_kwargs={"prompt": PROMPT} # 5단계에서 정의한 커스텀 프롬프트
)

print("✓ RAG 체인 구성 완료!")
print("이제 질문에 답변할 수 있습니다.")

✓ RAG 체인 구성 완료!
이제 질문에 답변할 수 있습니다.


In [ ]:
# ============================================
# 실행: 질문에 대한 답변 생성
# ============================================

# 1. 질문 입력
question = "이 문서의 주요 내용은 무엇인가요?"

print(f"질문: {question}\n")
print("답변 생성 중...\n")

# 2. RAG 체인 실행
# .invoke()를 사용하여 질문을 처리하고 결과를 받습니다.
result = qa_chain.invoke({"query": question})

# 3. 답변 출력
print("=" * 60)
print("답변:")
print("=" * 60)
print(result["result"])

# 4. 참고한 원본 문서(Source Documents) 출력
print("\n" + "=" * 60)
print(f"참고한 문서 수: {len(result.get('source_documents', []))}")
print("=" * 60)

if 'source_documents' in result:
    for i, doc in enumerate(result["source_documents"], 1):
        print(f"\n[문서 {i}]")
        # PyPDFLoader로 로드했다면 'page' 정보가 메타데이터에 포함됩니다.
        print(f"페이지: {doc.metadata.get('page', 'N/A')}")
        print(f"내용 미리보기: {doc.page_content[:200]}...")

질문: 이 문서의 주요 내용은 무엇인가요?

답변 생성 중...

답변:
- 대만 행정원이 AI 개발·응용을 촉진하기 위한 「AI 기본법」을 가결해 입법원에 제출했으며, 법안은 AI 개발 원칙, 정부의 역할·의무, 위험관리, 기술 혁신·국제협력, 지식재산권·개인정보 보호를 포괄함.
- 구글 리서치가 과학 문제 검증에 필요한 실증 소프트웨어를 자동 생성하는 AI 시스템을 개발했으며, 문제 설명·평가지표·데이터셋을 입력받아 최적화된 코드 솔루션을 도출하고 다양한 분야에서 기존 최고 성능을 뛰어넘는 결과를 달성함.

참고한 문서 수: 3

[문서 1]
페이지: 4
내용 미리보기: 정책･법제기업･산업기술･연구인력･교육
3
대만 행정원, AI 기본법 통과 후 입법원에 제출n대만 행정원이 AI 개발과 응용에 우호적 환경을 조성해 대만을 글로벌 AI 개발의 핵심 거점으로 만들기 위한 「AI 기본법」을 가결하고 입법원에 제출n법안은 AI 개발과 응용의 기본 원칙과 정부의 역할과 의무, AI 위험관리 방안을 포괄하며, 기술 혁신과 국제협력에 ...

[문서 2]
페이지: 4
내용 미리보기: 정책･법제기업･산업기술･연구인력･교육
3
대만 행정원, AI 기본법 통과 후 입법원에 제출n대만 행정원이 AI 개발과 응용에 우호적 환경을 조성해 대만을 글로벌 AI 개발의 핵심 거점으로 만들기 위한 「AI 기본법」을 가결하고 입법원에 제출n법안은 AI 개발과 응용의 기본 원칙과 정부의 역할과 의무, AI 위험관리 방안을 포괄하며, 기술 혁신과 국제협력에 ...

[문서 3]
페이지: 22
내용 미리보기: 정책･법제기업･산업기술･연구인력･교육
21
구글, 과학적 발견을 위한 실증 소프트웨어 생성 AI 시스템 개발n구글 리서치가 실험을 통해 확보한 증거를 활용해 해결할 수 있는 과학 문제 검증에 필요한 실증 소프트웨어를 생성하는 AI 시스템을 개발n과학 문제에 대한 설명과 평가 지표, 데이터셋을 입력받은 AI 시스템은 최적화 코드 솔루션을 도출하며, 실제 과학...
